In [1]:
# Import necessary libraries
import asyncio
import sys
import os
import warnings
from devtools import pprint
# Add the src directory to the Python path and suppress warnings
sys.path.append(os.path.abspath('../../..'))
warnings.filterwarnings("ignore", category=FutureWarning)


# Import MCP components
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# Set up the model

#model = ChatOpenAI(model="gpt-4o")
from dotenv import load_dotenv
load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.getenv("GEMINI_API_KEY"))

from devtools import pprint
from langchain_core.messages import HumanMessage

In [ ]:
import logging
from src.agent.market_intelligence_agent.config import TEAM_MEMBERS
from src.agent.market_intelligence_agent.graph import build_graph

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Default level is INFO
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)


def enable_debug_logging():
    """Enable debug level logging for more detailed execution information."""
    logging.getLogger("src").setLevel(logging.DEBUG)


logger = logging.getLogger(__name__)

# Create the graph
graph = build_graph()


def run_agent_workflow(user_input: str, debug: bool = False):
    """Run the agent workflow with the given user input.

    Args:
        user_input: The user's query or request
        debug: If True, enables debug level logging

    Returns:
        The final state after the workflow completes
    """
    if not user_input:
        raise ValueError("Input could not be empty")

    if debug:
        enable_debug_logging()

    logger.info(f"Starting workflow with user input: {user_input}")
    result = graph.invoke(
        {
            # Constants
            "TEAM_MEMBERS": TEAM_MEMBERS,
            # Runtime Variables
            "messages": [{"role": "user", "content": user_input}],
            "deep_thinking_mode": True,
        }
    )
    logger.debug(f"Final workflow state: {result}")
    logger.info("Workflow completed successfully")
    return result


if __name__ == "__main__":
    print(graph.get_graph().draw_mermaid())


In [6]:
import sys
user_query = ["what is the latest news about tarrify between US and China? limit you query within recent 3 days"]


result = run_agent_workflow(user_input=user_query, debug=True)

# Print the conversation history
print("\n=== Conversation History ===")
for message in result["messages"]:
    role = message.type
    print(f"\n[{role.upper()}]: {message.content}")

2025-04-16 10:33:00,674 - __main__ - INFO - Starting workflow with user input: ['what is the latest news about tarrify between US and China? limit you query within recent 3 days']
2025-04-16 10:33:00,675 - src.agent.market_intelligence_agent.graph.nodes - INFO - Coordinator talking.
2025-04-16 10:33:00,808 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 400 Bad Request"


BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'Value is not a struct: "what is the latest news about tarrify between US and China? limit you query within recent 3 days"', 'status': 'INVALID_ARGUMENT'}}]